In [ ]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

In [ ]:
mnist = input_data.read_data_sets("./data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

In [ ]:
learning_rate = 0.001
training_epochs = 15
num_classes = 10
batch_size = 100

In [ ]:
element_size = 28
time_steps = 28
hidden_layer_size = 128

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name='X')
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='Y')

In [ ]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
#rnn_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size)
#rnn_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)

In [ ]:
W = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], stddev=0.01))
b = tf.Variable(tf.truncated_normal([num_classes], stddev=0.01))

In [ ]:
last_rnn_output = outputs[:,-1,:]
logits = tf.matmul(last_rnn_output, W) + b

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# initialize
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())

In [ ]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    total_batch = int(mnist.train.num_examples / batch_size)
    total_batch_test = int(mnist.test.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        acc, c, _ = sess.run([accuracy, cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    for i in range(total_batch_test):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)        
        batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_test_acc += acc / total_batch_test

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 
          'train accuracy = ', '{:.5f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.5f}'.format(avg_test_acc))


print('Learning Finished!')